In [ ]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import alpha_vantage as av
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.techindicators import TechIndicators as ti
from alpha_vantage.timeseries import TimeSeries as ts
import panel as pn
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
from holoviews import opts
# from holoviews.plotting.links import RangeToolLink

In [ ]:
load_dotenv()

In [ ]:
# Specify paper trading environment & set API keys
alpaca_paper_endpoint=os.getenv("ALPACA_PAPER_ENDPOINT")
alpaca_api_key=os.getenv("ALPACA_API_KEY")
alpaca_secret_key=os.getenv("ALPACA_SECRET_KEY")

# Create alpaca API object
alpaca_api=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    alpaca_paper_endpoint,
    api_version='v2'
)

# List Positions
positions=alpaca_api.list_positions()

# Put positions and extra info into DataFrame
positions_df=pd.DataFrame({
    'exchange':[x.exchange for x in positions],
    'ticker':[x.symbol for x in positions],
    'qty':[x.qty for x in positions],
    'lastday_price':[x.lastday_price for x in positions],
    'current_price':[x.current_price for x in positions],
    'change_today':[x.change_today for x in positions],
    'market_value':[x.market_value for x in positions],
    'unrealized_intraday_pl':[x.unrealized_intraday_pl for x in positions],
    'unrealized_intraday_plpc':[x.unrealized_intraday_plpc for x in positions],
})

# Export positions_df to .csv file
positions_df.to_csv('Data/stock_positions.csv')

# Create tickers list from positions_df DataFrame
# positions_tickers=positions_df.ticker.tolist()
positions_tickers=['AAPL','GOOG']

In [ ]:
# Setup AV API
av_api_key=os.getenv("ALPHAVANTAGE_API_KEY")
av_api='https://www.alphavantage.co/query?'

# Create technical indicators query
ti_query=ti(key=av_api_key,output_format='pandas')

# Create time-series query
ts_query=ts(key=av_api_key,output_format='pandas')

# Set query input variables
interval='60min'
period=60
outputsize='full'
series_type='close'

In [ ]:
# Set sleep interval (for testing with less indicators and tickers)
sleep=12

In [ ]:
# Initiate tabs
tabs=pn.Tabs()

# Loop through positions_tickers to pull API data into Dataframes
for x in positions_tickers:
    
    # Get RSI
    print(f"Getting RSI for {x} ticker")
    rsi_df,meta_data_rsi=ti_query.get_rsi(symbol=x,
                                          interval=interval,
                                          time_period=period,
                                          series_type=series_type,
    )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)

    # Get SMA
    print(f"Getting SMA for {x} ticker")
    sma_df,meta_data_sma=ti_query.get_sma(symbol=x,
                                          interval=interval,
                                          time_period=period,
                                          series_type=series_type,
    )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)

    # Get EMA
    print(f"Getting EMA for {x} ticker")
    ema_df,meta_data_ema=ti_query.get_ema(symbol=x,
                                          interval=interval,
                                          time_period=period,
                                          series_type=series_type,
    )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)
    
    
    # Get BBANDS
    print(f"Getting BBANDS for {x} ticker")
    bbands_df,meta_data_bbands=ti_query.get_bbands(symbol=x,
                                                   interval=interval,
                                                   time_period=period,
                                                   series_type=series_type,
    )
    # Rename BBANDS columns
    bbands_df=bbands_df.rename(columns={'Real Upper Band':'Upper BBand',
                                        'Real Lower Band':'Lower BBand',
                                        'Real Middle Band':'Middle BBand'}
                              )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep) 

    # Get MACD
    print(f"Getting MACD for {x} ticker")
    macd_df,meta_data_macd=ti_query.get_macd(symbol=x,
                                             interval=interval,
                                             series_type=series_type,
    )
    #Drop MACD_Hist column and rename MACD_Signal to 'MACD Signal'
    macd_df=macd_df.drop(['MACD_Hist'],axis=1).rename(columns={'MACD_Signal':'MACD Signal'})
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)

    # Get Time-Series
    print(f"Getting time-series for {x} ticker")
    data_ts,meta_data_ts=ts_query.get_intraday(symbol=x,
                                               interval=interval,
                                               outputsize=outputsize,
    )
    # Create time-series DataFrame
    print(f"Creating time-series DataFrame for {x} ticker")
    ts_df=pd.DataFrame.from_dict(data_ts)
    ts_df=ts_df.drop(['5. volume','3. low','2. high','1. open'],axis=1)
    ts_df=ts_df.rename(columns={'4. close':'Closing Price'})
    ts_df=ts_df[(ts_df.index.get_level_values(0) >= '2015-07-16 07:00:00') & (df.index.get_level_values(0) <= '2015-07-16 23:00:00')]
    
    past_month=datetime.now().date() - timedelta(days=datetime.now().weekday()) - timedelta(weeks=-4)
    # Merge DataFrames as needed for plots and output combined data for ticker to .csv file    
    print(f'Merging all data for {x} and outputting to a .csv file')
    ticker_indicator_df=pd.concat([bbands_df,ema_df,rsi_df,sma_df,macd_df,ts_df],axis='columns',join='inner')
    ticker_indicator_df.to_csv('Data/'+f'{x}'+'_data.csv')#, index = False)
    
        # Set tart dates:
        start_date = df.index[-1] - pd.tseries.offsets.
    ts_mvg_avg_df=pd.concat([ema_df,sma_df,ts_df],axis='columns',join='inner')
    ts_bbands_df=pd.concat([bbands_df,ts_df],axis='columns',join='inner')

# Loop to plot each tickers incidators over time-series data
    for t in x:  
        # Create Time-Series/MVG_AVG chart
        print(f'Creating Time-Series/Mvg Average chart for {x}')
        ts_mvg_avg_plot=ts_mvg_avg_df.hvplot(kind='line',
                                             ylabel='Values',
                                             legend='top_left',
                                             title=f'Time-Series Plot for {x} Symbol',
                                             width=725,
                                             height=350,
        )

        # Create Time-Series/BBANDS chart
        print(f'Creating Time-Series/BBANDS chart for {x}')      
        ts_bbands_plot=ts_bbands_df.hvplot(kind='line',
                                           ylabel='Values',
                                           legend=('top_left'),
#                                            rot=90,
                                           title=f'Time-Series Plot for {x} Symbol',
                                           width=725,
                                           height=350,
#                                            legend_title='',
        )

        # Create MACD chart
        print(f'Creating MACD chart for {x}') 
        macd_plot=macd_df.hvplot(kind='line',
                                 x='date',
                                 xlabel='Date',
                                 ylabel='MACD',
                                 legend='top_left',
                                 width=725,
                                 height=150, 
        )

        # Create RSI chart
        print(f'Creating RSI chart for {x}') 
        rsi_plot=rsi_df.hvplot(kind='line',
                               x='date',
                               xlabel='Date',
                               ylabel='RSI',
                               width=725,
                               height=150,
        )
        
        mvg_avg_layout=(ts_mvg_avg_plot+macd_plot).cols(1)
        mvg_avg_layout.opts(opts.Layout(shared_axes=False,merge_tools=False))
        
        bollinger_layout=(ts_bbands_plot+rsi_plot).cols(1)
        bollinger_layout.opts(opts.Layout(shared_axes=True,merge_tools=False))
        
        # Save chart as .html file
        hvplot.save(layout,('Data_Analysis/'+f'{x}_Interactive_Chart.html')
        # Save chart as .png file
        background = hv.RGB.load_image('/Data/oil_and_gas.png')
        ts_mvg_avg_plot.save(background,'f'{x}'_plot.png', fmt='svg')
        penguins
    # Create dashboard panel
    mvg_avg_col=pn.Column(mvg_avg_layout)
    bollinger_col=pn.Column(bollinger_layout)
    tab_row=pn.Row(mvg_avg_col,bollinger_col)
    
    # Put the panel in a tab
    tabs.append((f'{x}',tab_row),)
    
#     # Sleep
#     print(f"Sleeping for {sleep} seconds before moving to next ticker...")
#     time.sleep(sleep)   

# Serve the dashboard
print('Complete... Displaying dashboard in browser now')
pn.serve(title='Portfolio Technical Indicators',panels=tabs)